In [ ]:
%%capture
!git clone https://github.com/gunnxx/indonesian-mt-data
!pip3 install transformers datasets sacrebleu sentencepiece

In [ ]:
import datasets
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = 'cuda'
sacrebleu = datasets.load_metric("sacrebleu")
tokenizer = AutoTokenizer.from_pretrained("Wikidepia/marian-nmt-enid")
model = AutoModelForSeq2SeqLM.from_pretrained("Wikidepia/marian-nmt-enid").to(device)

In [ ]:
from glob import glob

test = {}
def read_references(paths: list, src: str, trg: str):
  assert len(paths) == 2
  ret = {"src": [], "trg": []}
  src_file = open(paths[0] if paths[0].endswith(src) else paths[1])
  trg_file = open(paths[0] if paths[0].endswith(trg) else paths[1])
  for ref_src, ref_trg in zip(src_file, trg_file):
    ret["src"].append(ref_src.strip())
    ret["trg"].append(ref_trg.strip())
  return ret

for domain_folder in glob("indonesian-mt-data/*/"):
  test_files = glob(domain_folder + "valid.*")
  domain = domain_folder.split("/")[1]
  test[domain] = read_references(test_files, "en", "id")

In [ ]:
dataset = datasets.Dataset.from_dict(test["general"])

In [ ]:
def evaluate(batch):
  inputs = tokenizer.prepare_seq2seq_batch(batch["src"], return_tensors="pt", padding=True).to(device)
  pred_ids = model.generate(**inputs)
  batch["pred"] = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  return batch

In [ ]:
result = dataset.map(evaluate, batched=True, batch_size=12)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3226: FutureWarning: `prepare_seq2seq_batch` is deprecated and will be removed in version 5 of 🤗 Transformers. Use the regular `__call__` method to prepare your inputs and the tokenizer under the `with_target_tokenizer` context manager to prepare your targets. See the documentation of your specific tokenizer for more details
  FutureWarning,


In [ ]:
references = []
for src, trg in zip(result["src"], result["trg"]):
  references.append([src, trg])

In [ ]:
sacrebleu.compute(predictions=result["pred"], references=references)

{'bp': 0.9751209167935732,
 'counts': [22549, 12002, 6870, 4090],
 'precisions': [63.33277159869677,
  35.66610204748744,
  21.673291690327467,
  13.749285642249639],
 'ref_len': 36501,
 'score': 27.93063185245723,
 'sys_len': 35604,
 'totals': [35604, 33651, 31698, 29747]}